In [2]:
import { TextLoader } from "langchain/document_loaders/fs/text";
  
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();
console.log(docs);

[
  Document {
    pageContent: "三体前传：球状闪电 作者：刘慈欣\n" +
      "\n" +
      "内容简介：\n" +
      "　　没有《球状闪电》，就没有后来的《三体》！\n" +
      "　　《三体》前传！\n" +
      "　　亚洲首位雨果奖得主刘慈欣的三大长篇之一！（《三体》《球状闪电》《超新星纪元》）\n" +
      "　"... 189395 more characters,
    metadata: { source: "data/qiu.txt" }
  }
]


In [6]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { TextLoader } from "langchain/document_loaders/fs/text";
  
const loader = new TextLoader("data/qiu.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100,
});

const splitDocs = await splitter.splitDocuments(docs);

console.log(splitDocs[4].pageContent)


序曲
　　今天是我的生日，直到晚上爸爸妈妈点上了生日蛋糕的蜡烛，我们三个围着十四个小火苗坐下来，我才想起这事。
　　这是个雷雨之夜，整个宇宙似乎是由密集的闪电和我们的小屋组成。当那蓝色的电光闪起时，窗外的雨珠在一瞬间看得清清楚楚，那雨珠似乎凝固了，像密密地挂在天地间的一串串晶莹的水晶。这时我的脑海中就有一个闪念：世界要是那样的也很有意思，你每天一出门，就在那水晶的密帘中走路，它们在你周围发出丁零丁零的响声，只是，这样玲珑剔透的世界，如何经得住那暴烈的雷电呢……世界在我的眼中总和在别人眼中不一样，我总是努力使世界变形，这是我长这么大对自己唯一的认识。
　　暴雨是从傍晚开始的，自那以后闪电和雷声越来越密，开始，每当一道闪电过后，我脑海中一边回忆着刚才窗外那转瞬即逝的水晶世界，一边绷紧头皮等待着那一声炸雷，但现在，闪电太密集了，我已分不出哪声雷属于哪个闪电了。


In [8]:
import { load } from "dotenv";
const env = await load();
const process = {
    env
}

In [9]:
import { AlibabaTongyiEmbeddings } from "@langchain/community/embeddings/alibaba_tongyi";
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const embeddings = new AlibabaTongyiEmbeddings();


const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

const retriever = vectorstore.asRetriever(2);

const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论");

console.log(res);


[retriever/start] [1:retriever:VectorStoreRetriever] Entering Retriever run with input: {
  "query": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[retriever/end] [1:retriever:VectorStoreRetriever] s] Exiting Retriever run with output: {
  "documents": [
    {
      "pageContent": "“那么，像刚才说的，是否存在宏质子和宏中子呢？”\n　　“应该存在，不过由于它们不能被激发，我们很难发现它们。”\n　　“丁教授，你的梦实现了。”林云说，除了丁仪和我，别的人还不太明白她这话的意思。\n　　“是啊是啊，真有西瓜这么大的基本粒子摆上物理学家的桌面了，下一步我们肯定要研究它们的内部结构，那也是由弯曲的空间构成的结构，虽然也很难，但我相信比研究微观粒子的结构不知要容易多少倍。”\n　　“那也存在宏原子了？三种宏粒子应该是能够组成原子的啊！”\n　　“是的，应该有宏原子。”\n　　“我们所捕获到的那个空泡，哦，那个宏电子，是自由电子呢，还是一个宏原子中的电子？如果是后者，那这个宏原子的原子核在哪里呢？”\n　　“呵呵，您问住我了。不过，原子中的空间很大，如果一个原子有一个剧场大厅那么大，原子核只是大厅中央的一个核桃大小，所以，如果这个宏电子真的属于一个宏原子，那它的原子核距离我们是相当远的。”\n　　“天啊，还有一个大问题：如果存在宏原子，那一定有宏物质，也有宏世界了？”\n　　“我们已经在进行宏伟的哲学思考了。”丁仪向提问者微笑着说。",
      "metadata": {
        "source": "data/qiu.txt",
        "loc": {
          "lines": {
            "from": 1465,
            "to": 1474
          }
        }
      }
    },
    {
      "pageContent": "“宏原子核？”\n　　“是的，通过观测一个宏电子在空间中的运动，借助这个数学模型

In [11]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (documents: Document[]): string => {
    return documents.map((document) =>  document.pageContent).join("\n")
};

const contextRetriverChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    convertDocsToString
]);

const result = await contextRetriverChain.invoke({ question: "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"});

console.log(result);


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input: {
  "question": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableLambda] Entering Chain run with input: {
  "question": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[chain/start] [1:chain:RunnableLambda] Entering Chain run with input: {
  "question": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableLambda] s] Exiting Chain run with output: {
  "output": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[chain/end] [1:chain:RunnableLambda] s] Exiting Chain run with output: {
  "output": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[retriever/start] [1:chain:RunnableSequence > 3:retriever:VectorStoreRetriever] Entering Retriever run with input: {
  "query": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[retriever/start] [1:retriever:VectorStoreRetriever] Entering Retriever run with input: {
  "query": "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论"
}
[retriever/end] [1:chain:RunnableSequence 

In [12]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(
    TEMPLATE
);


In [13]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const model = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
});

const ragChain = RunnableSequence.from([
    {
        context: contextRetriverChain,
        question: (input) => input.question,
    },
    prompt,
    model,
    new StringOutputParser()
])


In [15]:
process.env.LANGCHAIN_VERBOSE = "false";
const answer = await ragChain.invoke({
    question: "什么是球状闪电"
});  

console.log(answer);

球状闪电是一种奇特的自然现象，它在书中被描述为一种能量高度集中、外形类似球体的闪电，其特性以真实历史记录为依据。在小说中，球状闪电不仅具有神秘性，还被应用于科研项目中，如用于医疗领域，用以精确地烧掉癌细胞而不影响周围正常组织。研究者们关注的是球状闪电在没有观察者的情况下是否仍然保持坍缩状态，这是他们项目中的一个超越现实意义的探索。


In [16]:
const answer = await ragChain.invoke({
    question: "详细描述原文中有什么跟直升机相关的场景"
});

console.log(answer);

在《球状闪电》中，有关直升机的场景描述非常生动。首先，两架直升机按照计划升至预定高度，它们在顺风中飞行，尽量减小相对气流速度，然后点燃电弧进行试验。这个过程中，它们之间的距离逐渐缩小，电弧从最初的一点明亮弧光变成了一条在风中狂舞的蓝紫色弧光，非常壮观。电弧熄灭的瞬间，悲剧发生了，机尾的小螺旋桨处出现火光，随后是白烟和失控的旋转。由于闪电击毁了尾桨的控制线路，直升机失去了尾桨的平衡，机体开始自旋并逐渐失去升力，最终摇晃着坠落。许大校连续两次通过无线电催促飞行员跳伞，但尝试重启尾桨的努力未能阻止直升机的坠落。整个过程充满了紧张与危机。
